## This notebook demonstrates how to remove the bias during "Pre-processing stage" using AI 360 fairness toolkit by adjusting the weights

### Pre-processing algorithm
A bias mitigation algorithm that is applied to training data.

### Insert your credentials as credentials in the below cell
Click on dropdown from Pipeline_LabelEncoder-0.1.zip under Data tab and select 'Credentials'

In [ ]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials = {
    
}


In [ ]:
from ibm_botocore.client import Config
import ibm_boto3

cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])

In [ ]:
import os
os.getcwd()

In [ ]:
cos.download_file(Bucket=credentials['BUCKET'],Key='Pipeline_LabelEncoder-0.1.zip',Filename='/home/wsuser/work/Pipeline_LabelEncoder-0.1.zip')

In [ ]:
!ls

In [ ]:
!pip install Pipeline_LabelEncoder-0.1.zip
!pip install aif360

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
!pip install 'tensorflow>=2'

In [ ]:
import pandas as pd
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.datasets import BinaryLabelDataset
from IPython.display import Markdown, display
from aif360.algorithms.preprocessing.reweighing import Reweighing
import warnings
warnings.filterwarnings("ignore")

### Insert the data as Pandas Dataframe and change the name from df_data_ to df

In [ ]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_abe73f625ce04410993f962955e934a8 = ''
else:
    endpoint_abe73f625ce04410993f962955e934a8 = ''

client_abe73f625ce04410993f962955e934a8 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='',
    ibm_auth_endpoint="",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_abe73f625ce04410993f962955e934a8)

body = client_abe73f625ce04410993f962955e934a8.get_object(Bucket='wwcdemo-donotdelete-pr-t7c1vptc0iudk2',Key='fraud_data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


In [ ]:
df.describe(include = 'all')

In [ ]:
privileged_groups = [{'Gender': 1}]
unprivileged_groups = [{'Gender': 0}]
favorable_label = 1 
unfavorable_label = 0

In [ ]:
from sklearn import preprocessing
categorical_column = ['Gender', 'Married', 'Fraud_risk']

data_encoded = df.copy(deep=True)
#Use Scikit-learn label encoding to encode character data
lab_enc = preprocessing.LabelEncoder()
for col in categorical_column:
        data_encoded[col] = lab_enc.fit_transform(df[col])
        le_name_mapping = dict(zip(lab_enc.classes_, lab_enc.transform(lab_enc.classes_)))
        print('Feature', col)
        print('mapping', le_name_mapping)
        

data_encoded.head()

In [ ]:
from Pipeline_LabelEncoder.sklearn_label_encoder import PipelineLabelEncoder
preprocessed_data = PipelineLabelEncoder(columns = ['Gender','Married', 'Fraud_risk']).fit_transform(data_encoded)
print('-------------------------')
#print('validation data encoding')
#validation_enc_data = PipelineLabelEncoder(columns = ['Gender','Married', 'Fraud_risk']).transform(validation_input_data)

In [ ]:
#Create binary label dataset that can be used by bias mitigation algorithms
fraud_dataset = BinaryLabelDataset(favorable_label=favorable_label,
                                unfavorable_label=unfavorable_label,
                                df=preprocessed_data,
                                label_names=['Fraud_risk'],
                                protected_attribute_names=['Gender', 'Married'],
                                unprivileged_protected_attributes=unprivileged_groups)

In [ ]:
display(Markdown("#### Training Data Details"))
print("shape of the training dataset", fraud_dataset.features.shape)
print("Training data favorable label", fraud_dataset.favorable_label)
print("Training data unfavorable label", fraud_dataset.unfavorable_label)
print("Training data protected attribute", fraud_dataset.protected_attribute_names)
print("Training data privileged protected attribute (1:Male and 0:Female)", 
      fraud_dataset.privileged_protected_attributes)
print("Training data unprivileged protected attribute (1:Male and 0:Female)",
      fraud_dataset.unprivileged_protected_attributes)

In [ ]:
metric_orig_train = BinaryLabelDatasetMetric(fraud_dataset, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % 
      metric_orig_train.mean_difference())

In [ ]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
               privileged_groups=privileged_groups)
RW.fit(fraud_dataset)
train_tf_dataset = RW.transform(fraud_dataset)

In [ ]:
train_tf_dataset.labels

In [ ]:
metric_orig_train = BinaryLabelDatasetMetric(train_tf_dataset, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Difference in mean outcomes between unprivileged and privileged groups = %f"
      % metric_orig_train.mean_difference())

## There's no unfair advantage between unprivileged and privileged groups